In [1]:
import pandas as pd
import seaborn as sb
import pylab as py
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import folium
from folium import plugins
from geopy.geocoders import Nominatim
import datetime
import statsmodels.api as sm
import statsmodels.stats as st
import scipy.stats as sp
from statsmodels.graphics.gofplots import ProbPlot
from statsmodels.stats.diagnostic import het_goldfeldquandt

In [2]:
df = pd.read_csv('microplastics-in-ocean.csv')
df.head()

,Year,Accumulated ocean plastic: Microplastics (<0.5cm),Yearly Microplastics Emission
0,1950,0,0
1,1951,0,0
2,1952,0,0
3,1953,0,0
4,1954,100,100


In [3]:
df = df.loc[(df['Year'] >= 1974) & (df['Year'] <= 2004)]
df.insert(0, 'Time', range(1, 1 + len(df)))
df = df.drop(["Accumulated ocean plastic: Microplastics (<0.5cm)"], axis = 1)
df.head()

,Time,Year,Yearly Microplastics Emission
24,1,1974,1400
25,2,1975,1500
26,3,1976,1800
27,4,1977,2000
28,5,1978,2200


In [4]:
df_female = pd.read_csv("female-pop.csv")
df_female = df_female.groupby("Year").sum()
df_female = df_female.reset_index()
df_female = df_female.loc[(df_female['Year'] >= 1980) & (df_female['Year'] <= 2010)]
df_female = df_female.reset_index()
df_female = df_female.drop(columns = ["index"])
df_female.head()

,Year,PopFemale
0,1980,8.682137e+07
1,1981,8.839908e+07
2,1982,9.000685e+07
3,1983,9.165150e+07
4,1984,9.334087e+07


In [5]:
df_cancer = pd.read_csv('cancer.csv')
df_cancer = df_cancer.loc[(df_cancer['Year'] >= 1980) & (df_cancer['Year'] <= 2010)]
df_cancer['Cancer Rate'] = df_cancer.sum(axis=1) - df_cancer["Year"]
df_cancer = df_cancer.reset_index()
df_cancer = df_cancer.drop(columns = ["index"])
df_cancer = pd.concat([df_cancer['Year'], df_cancer['Cancer Rate']], axis = 1)
df_cancer["Cancer Rate"] = df_cancer["Cancer Rate"]/ df_female["PopFemale"] *100
df_cancer.head()

,Year,Cancer Rate
0,1980,0.069059
1,1981,0.069346
2,1982,0.075905
3,1983,0.090922
4,1984,0.092270


In [6]:
len(df_cancer)

31

In [7]:
female_df = pd.read_csv('female-bmi.csv')
female_df = female_df.loc[(female_df['Year'] >= 1975) & (female_df['Year'] <= 2005)]
female_df = female_df.groupby(['Year']).median()
female_df = female_df.reset_index()

In [8]:
alcohol_df = pd.read_csv('alcohol.csv')
alcohol_df = alcohol_df.loc[(alcohol_df['Year'] >= 1972) & (alcohol_df['Year'] <= 2002)]
alcohol_df = alcohol_df.groupby("Year").sum()
alcohol_df = alcohol_df.reset_index()
alcohol_df = alcohol_df.drop(columns = "Year")
alcohol_df["AlcAmt"] = alcohol_df["AlcAmt"]/2
alcohol_df.head()

,AlcAmt
0,747.230
1,784.310
2,787.020
3,796.570
4,796.565


In [9]:
smoke_df = pd.read_csv('smoke.csv')
smoke_df = smoke_df.loc[(smoke_df['Year'] >= 1972) & (smoke_df['Year'] <= 2002)]
smoke_df = smoke_df.groupby("Year").median()
smoke_df = smoke_df.reset_index()
smoke_df = smoke_df.drop(columns = "Year")
smoke_df.head()

,Cig/Adult/Day
0,6.60
1,7.15
2,7.40
3,6.95
4,7.35


In [10]:
age_df = pd.read_csv('age.csv')
age_df = age_df.loc[(age_df['Year'] >= 1980) & (age_df['Year'] <= 2010)]
age_df = age_df.reset_index()
age_df = age_df.drop(columns = ["Year", "index"])
age_df.head()         

,MedianAge
0,23.724996
1,23.907336
2,24.089675
3,24.272015
4,24.454354


In [ ]:
y.head()

In [ ]:
X.head()

In [ ]:
fig,axs = plt.subplots(2,3, figsize = (15,15))

axs[0,0].boxplot(x = X['Yearly Microplastics Emission'], vert = False)
axs[0,1].boxplot(x = X['Mean BMI (female)'], vert = False)
axs[0,2].boxplot(x = X['AlcAmt'], vert = False)
axs[1,0].boxplot(x = X['Cig/Adult/Day'], vert = False)
axs[1,1].boxplot(x = X['MedianAge'], vert = False)
axs[1,2].boxplot(x = y['Cancer Rate'], vert = False)

axs[0,0].set_title("MPP")
axs[0,1].set_title("BMI")
axs[0,2].set_title("AlcAmt")
axs[1,0].set_title("Cigarettes")
axs[1,1].set_title("MedianAge")
axs[1,2].set_title("Cancer")

In [ ]:
sp.kstest(X['AlcAmt'],'norm')

In [ ]:
mpp = st.diagnostic.kstest_normal(X['Yearly Microplastics Emission'])[1]
bmi = st.diagnostic.kstest_normal(np.power(X['Mean BMI (female)'],100))[1]
alc = st.diagnostic.kstest_normal(X['AlcAmt'])[1]
cig = st.diagnostic.kstest_normal(X['Cig/Adult/Day'])[1] 
age = st.diagnostic.kstest_normal(np.power(X['MedianAge'],100))[1]
cancer = st.diagnostic.kstest_normal(y['Cancer Rate'])[1]


print("MPP: " + str(mpp))
print("BMI: " + str(bmi))
print("ALC: " + str(alc))
print("CIG: " + str(cig))
print("AGE: " + str(age))
print("CANCER: " + str(cancer))

In [ ]:
plt.hist(X['Yearly Microplastics Emission'])

In [ ]:
X = df
# X = X.reset_index().drop(columns = ["index","Year"])
X = X.reset_index().drop(columns = ["index","Year","Time"])

# bmi df refactoring
# female_df = female_df.reset_index().drop(columns = ["index", "level_0"]) 
female_df = female_df.reset_index()
 
# X = pd.concat([X, bmi_df, alcohol_df], axis = 1) 
# X = pd.concat([X, female_df, alcohol_df, smoke_df], axis = 1)
X = pd.concat([X, female_df, alcohol_df, smoke_df, age_df], axis = 1)
X = X.drop(columns = ["index", "Year"])

y = df_cancer['Cancer Rate']
y = y.reset_index().drop(columns = "index")
y = y.astype(float)

# Define the modell
X = sm.add_constant(X)
model = sm.OLS(y, X)

# Fit the model
model_fit = model.fit()

# Print the model summary
print(model_fit.summary())

In [ ]:
# model values
model_fitted_y = model_fit.fittedvalues
# model residuals
model_residuals = model_fit.resid
# normalized residuals
model_norm_residuals = model_fit.get_influence().resid_studentized_internal
# absolute squared normalized residuals
model_norm_residuals_abs_sqrt = np.sqrt(np.abs(model_norm_residuals))
# absolute residuals
model_abs_resid = np.abs(model_residuals)
# leverage, from statsmodels internals
model_leverage = model_fit.get_influence().hat_matrix_diag
# cook's distance, from statsmodels internals
model_cooks = model_fit.get_influence().cooks_distance[0]

plot_lm_1 = plt.figure()
plot_lm_1.axes[0] = sb.residplot(model_fitted_y, df.columns[-1], data=df,
                          lowess=True,
                          scatter_kws={'alpha': 0.5},
                          line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})

plot_lm_1.axes[0].set_title('Residuals vs Fitted')
plot_lm_1.axes[0].set_xlabel('Fitted values')
plot_lm_1.axes[0].set_ylabel('Residuals');

In [ ]:
QQ = ProbPlot(model_norm_residuals)
plot_lm_2 = QQ.qqplot(line='45', alpha=0.5, color='#4C72B0', lw=1)
plot_lm_2.axes[0].set_title('Normal Q-Q')
plot_lm_2.axes[0].set_xlabel('Theoretical Quantiles')
plot_lm_2.axes[0].set_ylabel('Standardized Residuals');
# annotations
abs_norm_resid = np.flip(np.argsort(np.abs(model_norm_residuals)), 0)
abs_norm_resid_top_3 = abs_norm_resid[:3]
for r, i in enumerate(abs_norm_resid_top_3):
    plot_lm_2.axes[0].annotate(i,
                               xy=(np.flip(QQ.theoretical_quantiles, 0)[r],
                                   model_norm_residuals[i]));

In [ ]:
plot_lm_3 = plt.figure()
plt.scatter(model_fitted_y, model_norm_residuals_abs_sqrt, alpha=0.5);
sb.regplot(model_fitted_y, model_norm_residuals_abs_sqrt,
              scatter=False,
              ci=False,
              lowess=True,
              line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8});
plot_lm_3.axes[0].set_title('Scale-Location')
plot_lm_3.axes[0].set_xlabel('Fitted values')
plot_lm_3.axes[0].set_ylabel('$\sqrt{|Standardized Residuals|}$');

# annotations
abs_sq_norm_resid = np.flip(np.argsort(model_norm_residuals_abs_sqrt), 0)
abs_sq_norm_resid_top_3 = abs_sq_norm_resid[:3]
for i in abs_norm_resid_top_3:
    plot_lm_3.axes[0].annotate(i,
                                 xy=(model_fitted_y[i],
                                     model_norm_residuals_abs_sqrt[i]));

In [ ]:
plot_lm_4 = plt.figure();
plt.scatter(model_leverage, model_norm_residuals, alpha=0.5);
sb.regplot(model_leverage, model_norm_residuals,
              scatter=False,
              ci=False,
              lowess=True,
              line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8});
plot_lm_4.axes[0].set_xlim(0, max(model_leverage)+0.01)
plot_lm_4.axes[0].set_ylim(-3, 5)
plot_lm_4.axes[0].set_title('Residuals vs Leverage')
plot_lm_4.axes[0].set_xlabel('Leverage')
plot_lm_4.axes[0].set_ylabel('Standardized Residuals');

# annotations
leverage_top_3 = np.flip(np.argsort(model_cooks), 0)[:3]
for i in leverage_top_3:
    plot_lm_4.axes[0].annotate(i,
                                 xy=(model_leverage[i],
                                     model_norm_residuals[i]));